In [1]:
import os
import pandas as pd
from sfimpute.preprocess import *

In [2]:
%load_ext autoreload
%autoreload 2

## Mouse Embryonic Stem Cells (25kb)

Data are downloaded from [https://zenodo.org/records/3735329](https://zenodo.org/records/3735329). The annotation file is `41586_2020_3126_MOESM2_ESM.xlsx`.

In [3]:
mESC_dire = "data_mESC_seqFISH"
ann_path = os.path.join(mESC_dire, "41586_2020_3126_MOESM2_ESM.xlsx")
data_path = os.path.join(mESC_dire, "DNAseqFISH+{}loci-E14-replicate{}.csv")
resol = "25kb"

ann = pd.read_excel(ann_path, sheet_name=f"{resol}_loci")
ann = ann.dropna().astype(int, errors="ignore")
ann.head()

,Region ID,Name,Channel,Chrom,Start,End,Chrom ID
0,1,chr1-HighRes-#1,3,chr1,135600000,135625000,1
1,2,chr1-HighRes-#2,3,chr1,135625000,135650000,1
2,3,chr1-HighRes-#3,3,chr1,135650000,135675000,1
3,4,chr1-HighRes-#4,3,chr1,135675000,135700000,1
4,5,chr1-HighRes-#5,3,chr1,135700000,135725000,1


In [4]:
# merge the 25kb data from two biological replicates
rep1_data = pd.read_csv(data_path.format(resol, 1))
rep1_data["rep"] = 1
rep2_data = pd.read_csv(data_path.format(resol, 2))
rep2_data["rep"] = 2
data = pd.concat([rep1_data, rep2_data], ignore_index=True, sort=False)
data = data[(data["labelID"]==0)|(data["labelID"]==1)]
data.head()

,fov,channel,cellID,regionID (hyb1-60),x,y,z,dot_intensity,chr1_intensity,chr2_intensity,...,chr14_intensity,chr15_intensity,chr16_intensity,chr17_intensity,chr18_intensity,chr19_intensity,chrX_intensity,chromID,labelID,rep
0,0,3,1,2,1718.431196,146.909289,11.282036,638,5036,54,...,60,31,44,14,20,26,9,1,0,1
1,0,3,1,3,1717.986519,146.171951,11.083011,738,4984,38,...,27,40,34,9,26,24,24,1,0,1
2,0,3,1,3,1604.490932,204.195855,11.725510,223,3188,49,...,4,57,7,37,26,40,3,1,1,1
3,0,3,1,5,1718.106776,146.904642,10.363083,775,3084,16,...,37,25,18,15,12,22,21,1,0,1
4,0,3,1,5,1600.751787,204.916308,13.133202,787,6151,22,...,5,54,45,21,32,10,7,1,1,1


The columns can be interpreted as follows:
1. `chromID` is the ID of the region of interest
2. `regionID` is the loci ID within each region of interest
3. The following columns together gives a unique label for each haploid:
    - `rep`: biological replicate ID
    - `fov`: field of view
    - `cellID`: cell ID
    - `labelID`: haploid id within each cell

Thus in `format_fish_data`, we use `["rep", "fov", "cellID", "labelID"]` as the unique identifier.

In [5]:
# renumber pos id -> starts from 1 with each imaging region
ann[data.columns[3]] = (ann["Region ID"]-1)%60 + 1
ann = ann.rename({"Chrom ID":"chromID"}, axis=1)

post_ann, post_data = format_fish_data(
    ann, data, 
    "chromID", data.columns[3], "Start", "End", 
    ["rep", "fov", "cellID", "labelID"], []
)
# convert to nm
post_data["x"] = post_data["x"]*103
post_data["y"] = post_data["y"]*103
post_data["z"] = post_data["z"]*250
post_data.head()

,region,haploid,pos,x,y,z
0,1,rep.1.fov.0.cellID.1.labelID.0,2,176998.413188,15131.656798,2820.508940
1,1,rep.1.fov.0.cellID.1.labelID.0,3,176952.611457,15055.710912,2770.752832
2,1,rep.1.fov.0.cellID.1.labelID.0,5,176964.997928,15131.178147,2590.770663
3,1,rep.1.fov.0.cellID.1.labelID.0,10,176700.084915,14608.560370,2968.756472
4,1,rep.1.fov.0.cellID.1.labelID.0,11,176699.491326,14581.245027,3056.693920


In [6]:
post_data = fill_missing_by_nan(post_data, post_ann)
post_data.head()
# post_data.to_csv("mESC_seqFISH_25kb_coor_wnan.txt", sep="\t", index=False)
# post_ann.to_csv("mESC_seqFISH_25kb_ann.txt", sep="\t", index=False)

,region,haploid,pos,x,y,z
0,1,rep.1.fov.0.cellID.1.labelID.0,1,NaN,NaN,NaN
1,1,rep.1.fov.0.cellID.1.labelID.0,2,176998.413188,15131.656798,2820.508940
2,1,rep.1.fov.0.cellID.1.labelID.0,3,176952.611457,15055.710912,2770.752832
3,1,rep.1.fov.0.cellID.1.labelID.0,4,NaN,NaN,NaN
4,1,rep.1.fov.0.cellID.1.labelID.0,5,176964.997928,15131.178147,2590.770663


## Mouse Embryonic Stem Cells (1Mb)

Since the original data did not provide haploid information, here [Jie](https://github.com/b2jia/jie) aligned data are used, which are downloaded from the 4DN data portal with ID `4DNFIS6MLXGA` and `4DNFIU73OR5W`.

The function `read_4dn_csv` is used to process the data and extract annotation file. The argument `trace_cols` are the names of each part in trace_id separated by `_`.

In [7]:
data = []
ids4DN = ["FIS6MLXGA", "FIU73OR5W"]
for i, id4DN in enumerate(ids4DN):
    data_path = os.path.join("4dn_data", f"4DN{id4DN}.csv")
    ann, d = read_4dn_csv(data_path, trace_cols=["fov", "cellID", "reg", "labelID"])
    d["rep"] = i + 1
    data.append(d)
data = pd.concat(data, sort=False, ignore_index=True)
data.head()

,spot_id,trace_id,x,y,z,chrom,chrom_start,chrom_end,cell_id,extra_cell_roi_id,fov,cellID,reg,labelID,pos,rep
0,1,0_1_1_0,174.119,17.179,2.392,chr1,3100000,3125000,0_1,0,0,1,1,0,1,1
1,2280,0_1_1_1,165.662,20.608,2.370,chr1,5000000,5025000,0_1,0,0,1,1,1,2,1
2,377654,0_1_1_0,173.890,16.337,2.679,chr1,5057518,5082518,0_1,0,0,1,1,0,3,1
3,8021,0_1_1_1,164.751,20.349,2.548,chr1,8800000,8825000,0_1,0,0,1,1,1,5,1
4,369275,0_1_1_0,174.509,15.857,2.649,chr1,9535448,9560448,0_1,0,0,1,1,0,6,1


In [8]:
post_ann, post_data = format_fish_data(
    ann, data,
    "chrom", "pos", "chrom_start", "chrom_end",
    ["rep", "fov", "cellID", "labelID"], []
)
post_data["x"] = post_data["x"]*1e3
post_data["y"] = post_data["y"]*1e3
post_data["z"] = post_data["z"]*1e3
post_data = fill_missing_by_nan(post_data, post_ann)
post_data.head()
# post_data.to_csv("mESC_seqFISH_1Mb_coor_wnan.txt", sep="\t", index=False)
# post_ann.to_csv("mESC_seqFISH_1Mb_ann.txt", sep="\t", index=False)

,region,haploid,pos,x,y,z
0,chr1,rep.1.fov.0.cellID.1.labelID.0,1,174119.0,17179.0,2392.0
1,chr1,rep.1.fov.0.cellID.1.labelID.0,2,NaN,NaN,NaN
2,chr1,rep.1.fov.0.cellID.1.labelID.0,3,173890.0,16337.0,2679.0
3,chr1,rep.1.fov.0.cellID.1.labelID.0,4,NaN,NaN,NaN
4,chr1,rep.1.fov.0.cellID.1.labelID.0,5,NaN,NaN,NaN


## Mouse Embryonic Stem Cells (5kb)

The dataset is downloaded from 4DN data portal with ID `4DNFI9KE6AII`.

In [9]:
data_path = os.path.join("4dn_data", f"4DNFI9KE6AII.csv")
ann, data = read_4dn_csv(data_path, ["cell_id", "allele"])
data.head()

,spot_id,trace_id,x,y,z,chrom,chrom_start,chrom_end,cell_id,allele,pos
0,0,11_129,NaN,NaN,NaN,chr3,34601078,34606078,11,129,1
1,1,11_129,NaN,NaN,NaN,chr3,34606078,34611078,11,129,2
2,2,11_129,NaN,NaN,NaN,chr3,34611078,34616078,11,129,3
3,3,11_129,NaN,NaN,NaN,chr3,34616078,34621078,11,129,4
4,4,11_129,26455.163278,22916.077797,5128.801069,chr3,34621078,34626078,11,129,5


In [10]:
ann_129, ann_cast = ann.copy(), ann.copy()
ann_129["allele"] = "129"
ann_cast["allele"] = "CAST"
ann = pd.concat([ann_129, ann_cast], sort=False, ignore_index=True)

post_ann, post_data = format_fish_data(
    ann, data,
    "allele", "pos", "chrom_start", "chrom_end",
    ["cell_id"], []
)
idx = post_data[(post_data["region"]=="129")&(post_data["pos"]==26)].index
post_data.loc[idx, ["x", "y", "z"]] = np.nan
post_data.head()
# post_data.to_csv("mESC_Sox2_coor_wnan.txt", sep="\t", index=False)
# post_ann.to_csv("mESC_Sox2_ann.txt", sep="\t", index=False)

## Mouse Brain Cells (1Mb)

The mouse brain cell seqFISH+ data is downloaded from Zenodo with the link https://zenodo.org/records/4708112.

In [11]:
data_dire = "data_mBCC_seqFISH"
ann_path = os.path.join(data_dire, "science.abj1966_table_s1.xlsx")
data_path = os.path.join(data_dire, "TableS7_brain_DNAseqFISH_1Mb_voxel_coordinates_2762cells.csv")
resol = "1Mb"

sname = resol[:-2] + "-" + resol[-2:] + " resolution"
ann = pd.read_excel(ann_path, sheet_name=sname)
ann = ann.dropna().astype(int, errors="ignore")

data = pd.read_csv(data_path)
data = data[data["labelID"]>=0]
data.head()

,finalcellID,replicateID,rep,fov,cellID,cluster label,channel,chromID,geneID,x,y,z,labelID,XistID
0,1,1,2,0,2,1,1,1,chr1-#8,59.649000,785.560000,15.011000,0,NaN
1,1,1,2,0,2,1,2,1,Cadm3,74.819603,766.897425,8.578265,0,NaN
2,1,1,2,0,2,1,2,1,Satb2,85.056603,793.549425,13.143265,0,NaN
3,1,1,2,0,2,1,2,1,Tnfsf18,63.548603,776.486425,11.592265,0,NaN
4,1,1,2,0,2,1,1,1,chr1-#87,64.633000,774.357000,9.699000,0,NaN


In [12]:
post_ann, post_data = format_fish_data(
    ann, data, 
    "chromID", "geneID", "Start", "End", 
    ["rep", "fov", "cellID", "labelID"], ["cluster label"]
)
# convert to nm
post_data["x"] = post_data["x"]*103
post_data["y"] = post_data["y"]*103
post_data["z"] = post_data["z"]*250
post_data = fill_missing_by_nan(post_data, post_ann)
post_data.head()
# post_data.to_csv("mBCC_seqFISH_1Mb_coor_wnan.txt", sep="\t", index=False)
# post_ann.to_csv("mBCC_seqFISH_1Mb_ann.txt", sep="\t", index=False)

,region,haploid,pos,x,y,z,cluster label
0,1,rep.2.fov.0.cellID.2.labelID.0,1,6038.272000,80979.63000,2775.250000,1
1,1,rep.2.fov.0.cellID.2.labelID.0,2,NaN,NaN,NaN,1
2,1,rep.2.fov.0.cellID.2.labelID.0,3,6212.095082,80859.57574,3219.566288,1
3,1,rep.2.fov.0.cellID.2.labelID.0,4,5826.813000,81474.23600,2752.750000,1
4,1,rep.2.fov.0.cellID.2.labelID.0,5,NaN,NaN,NaN,1
